*In general, women speak at a higher pitch—about an octave higher than men. \
An adult woman's average range is from 165 to 255 Hz, while a man's is 85 to 155 Hz*

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import librosa 
from glob import glob

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv(r"C:\Users\admin\Desktop\datasets\male and female voices\train\train\targets.tsv", sep='\t', \
                      names=['id', 'gender'])


print("Total number of labels: {}".format(df.shape[0]))
print("Number of male: {}".format(df[df.gender == 0].shape[0]))
print("Number of female: {}".format(df[df.gender == 1].shape[0]))
df.head()

Total number of labels: 13936
Number of male: 7620
Number of female: 6316


,id,gender
0,5d1f7e43366513a1d0a6ec5640c3dc24,1
1,9a701a4536a05b6610a590a9fe702ed8,1
2,cad0b8547008d1524c1a0e5fd51f9908,1
3,4bbe607e7dc95460e2cc1a6ee5f4dfa6,0
4,30fb32cba90b34af26f3f14f5d636805,0


In [6]:
audio_files = glob("male and female voices\\train\\*\\*.wav")
y, sr = librosa.load(audio_files[4])
y, _ = librosa.effects.trim(y, top_db=20)
f0 = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))[0]

In [7]:
f0

array([         nan, 133.09934981, 133.8703865 , 136.21039964,
       138.59131549, 137.79308825, 137.79308825, 137.79308825,
       151.13490122, 146.83238396, 145.98669166, 136.21039964,
       133.09934981, 133.09934981, 132.33275397, 131.57057339,
       130.81278265, 125.62905732, 124.18608453, 124.18608453,
       124.18608453, 123.47082531,          nan,          nan])

In [8]:
male, female = [], []
for index, row in df.iterrows():
    path = "male and female voices\\train\\train\\{}.wav".format(str(row['id']))
    y, sr = librosa.load(path)
    y, _ = librosa.effects.trim(y, top_db=20)
    f0 = librosa.pyin(y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'))[0]
    if row['gender'] == 0:
        if not np.isnan(np.nanmean(f0)):
            male.append(np.nanmean(f0))
    if row['gender'] == 1:
        if not np.isnan(np.nanmean(f0)):
            female.append(np.nanmean(f0))
    if index % 100 == 0:
        print(index)
    if index == 500:
        break

0


C:\Users\admin\AppData\Local\Temp\ipykernel_2360\513765442.py:8: RuntimeWarning: Mean of empty slice
  if not np.isnan(np.nanmean(f0)):


100
200
300
400
500


In [30]:
# average fundamental frequency of male and female voices
f0male = np.asarray(male).mean()
f0female = np.asarray(female).mean()
f0male, f0female

(146.38487923470598, 227.74656984636752)

In [31]:
false_m = 0
false_f = 0
for f in male:
    if f > (f0female + f0male) / 2:
        false_m += 1

for f in female:
    if f < (f0female + f0male) / 2:
        false_f += 1

print("false positive/false negative rate: ", false_m / len(male), false_f / len(female))
print("accuracy: {}".format(1 - (false_m + false_f) / (len(male) + len(female))))

false positive/false negative rate:  0.07749077490774908 0.12334801762114538
accuracy: 0.9016064257028112
